### Import/Load/Initialise Dependencies

In [ ]:
from pylsl import StreamInlet, resolve_stream
from pynput import mouse
from datetime import date
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('float_format', '{:f}'.format)

In [ ]:
streams = resolve_stream('type', 'EEG')

In [ ]:
def on_click(x, y, button, pressed):
    raw_eeg_data.append([timestamp,*sample,1])

### Define Variables, Initialise Mouse Listener, Record the Data

In [ ]:
timeout = 1 #seconds

In [ ]:
raw_eeg_data = []

listener = mouse.Listener(
    on_click=on_click)
listener.start()

inlet = StreamInlet(streams[0])
print('Recording began. To insert a marker, click left mouse button.')

cur_time = inlet.pull_sample()[1]

while True:
    sample, timestamp = inlet.pull_sample()
    raw_eeg_data.append([timestamp,*sample,0])
    
    if timestamp > cur_time + timeout:
        listener.stop()
        break

raw_eeg_data = sorted(raw_eeg_data, key=lambda x: x[0])

df = pd.DataFrame(data=raw_eeg_data)
df = df.drop_duplicates(0, keep='last')

print('Recording complete. Number of samples collected: ', len(df.index))

### Plot Recorded Data for Preview

In [ ]:
fig = plt.figure(figsize=(12, 6))
ax = plt.gca()

df.plot(kind='line', use_index=True, y=1, ax=ax, label='tp9')
df.plot(kind='line', use_index=True, y=2, ax=ax, label='af7')
df.plot(kind='line', use_index=True, y=3, ax=ax, label='af8')
df.plot(kind='line', use_index=True, y=4, ax=ax, label='tp10')

ax.legend(loc="upper left")

### Save data to CSV

In [ ]:
today = date.today().strftime('%d%m%Y')
file_name = '%s_%s_%s.csv' % ('p300', today, 'test')

In [ ]:
df.to_csv(file_name, index=False, header=['timestamp','tp9','af7','af8','tp10','aux','marker'])